### Import in the necessary libraries

In [ ]:
import cv2 as cv
import threading
import os
from time import sleep
import ipywidgets as widgets
from mission_lib import Movement
from event_name import EventName
from IPython.display import display

### Import and initialize the Arm Device

In [ ]:
import Arm_Lib
Arm = Arm_Lib.Arm_Device()
joints_0 = [90, 90, 90, 90, 90, 90]
Arm.Arm_serial_servo_write6_array(joints_0, 1000)

### Initialize the movement objects, even name handlers and the music handlers

In [ ]:
movement = Movement(Arm)
e = EventName()
move_speed = {"Slow": 1500,
             "Regular": 1000,
             "Fast": 500}

### Create the widgets 

In [ ]:
button_layout = widgets.Layout(width='200px', height='60px', align_self='center')
short_layout = widgets.Layout(width='200px', height='75px', align_self='center')

output = widgets.Output()

choose_movement = widgets.ToggleButtons(options=['Slow', 'Regular', 'Fast'], button_style='success',
                                        tooltips=['Description of slow', 'Description of regular', 'Description of fast'])

# Movement Widgets
pinch_button = widgets.Button(description='Pinch', button_style='success', layout=button_layout)

release_button = widgets.Button(description='Release', button_style='primary', layout=button_layout)

up_button = widgets.Button(description='Up', button_style='primary', layout=short_layout)

down_button = widgets.Button(description='Down', button_style='primary', layout=short_layout)

left_button = widgets.Button(description='Left', button_style='primary', layout=short_layout)

right_button = widgets.Button(description='Right', button_style='primary', layout=short_layout)

# Sound Widget

play_button = widgets.Button(description='Play Sound', button_style='success', layout=button_layout)

stop_button = widgets.Button(description='Stop Sound', button_style='success', layout=button_layout)

# Exit Widget
exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)

imgbox = widgets.Image(format='jpg', height=480, width=640, layout=widgets.Layout(align_self='auto'))

img_box = widgets.VBox([imgbox, choose_movement], layout=widgets.Layout(align_self='auto'))

Slider_box = widgets.VBox([pinch_button, release_button, play_button,stop_button, exit_button],
                          layout=widgets.Layout(align_self='auto'))
Move_box = widgets.VBox([up_button, down_button, left_button, right_button],
                          layout=widgets.Layout(align_self='auto'))

controls_box = widgets.HBox([img_box, Move_box, Slider_box], layout=widgets.Layout(align_self='auto'))
# ['auto', 'flex-start', 'flex-end', 'center', 'baseline', 'stretch', 'inherit', 'initial', 'unset']

### Create the event handlers for the widgets

In [ ]:
play_button.on_click(e.play_button_Callback)
stop_button.on_click(e.stop_button_Callback)
pinch_button.on_click(e.pinch_button_Callback)
release_button.on_click(e.release_button_Callback)
up_button.on_click(e.up_button_Callback)
down_button.on_click(e.down_button_Callback)
left_button.on_click(e.left_button_Callback)
right_button.on_click(e.right_button_Callback)
exit_button.on_click(e.exit_button_Callback)

### Create the main function camera()

In [ ]:
def camera():
    
    # Open camera
    capture = cv.VideoCapture(1)
   # Be executed in loop when the camera is opened normally 
    while True:
        try:
            _, img = capture.read()

            img = cv.resize(img, (640, 480))

            if e.action == 'Up':
                movement.move_up(move_speed[choose_movement.value])
                e.reset()
            if e.action == 'Down':
                movement.move_down(move_speed[choose_movement.value])
                e.reset()
            if e.action == 'Left':
                movement.move_left(move_speed[choose_movement.value])
                e.reset()
            if e.action == 'Right':
                movement.move_right(move_speed[choose_movement.value])
                e.reset()
            if e.action == 'Pinch':
                movement.move_pincher(move_speed[choose_movement.value])
                e.reset()
            if e.action == 'Release':
                movement.release_pincher(move_speed[choose_movement.value])
                e.reset()
            if e.action == 'Play Music':
                os.system('rostopic pub -1 /play_specific std_msgs/String "data: \'/root/scripts/sensor/arm_sounds/music_cari.mp3\'"')
                e.reset()
            if e.action == 'No Music':
                os.system('rostopic pub -1 /play_specific std_msgs/String "data: \'stop\'"')
                e.reset()
            if e.action == 'Exit':
                cv.destroyAllWindows()
                capture.release()
                break
            imgbox.value = cv.imencode('.jpg', img)[1].tobytes()
            sleep(0.25)
        except KeyboardInterrupt:capture.release()

### Execute the camera() function

In [ ]:
display(controls_box,output)
threading.Thread(target=camera, ).start()

### After exiting and finishing all the tasks, delete the robot arm object. 

In [ ]:
del Arm